In [4]:
# Import libraries
import datetime
import functools
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.options.display.max_colwidth = 1000
import seaborn as sns

In [14]:
# .csv path
hotels_path = "../data/clean_hotels_scraped_v2.csv"
trivago_path = "../data/train.csv"

# Create dataframe
trivago_df = pd.read_csv(trivago_path)
hotels_df = pd.read_csv(hotels_path, usecols = ["city", "country", "hotel_name", "rating", 
                                                "address", "popularity_rating", "locality", "price",
                                               "landmark", "URL"])

# Sanity check
hotels_df.head()

,city,country,hotel_name,rating,address,popularity_rating,locality,price,landmark,URL
0,Mutterstadt,Germany,RheinCity Hotel,4.0,"Zollhofstraße 11, Ludwigshafen, RP, 67059, Germany",4.0,Ludwigshafen,107.0,"['5.4 miles to mutterstadt, 3.3 miles to mannheim (mhg)']",https://www.hotels.com/ho804141952/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=1&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
1,Mutterstadt,Germany,NYX Hotel Mannheim by Leonardo Hotels,4.0,"F4, 4-11, Mannheim, BW, 68159, Germany",14.0,Mannheim,78.0,"['6.2 miles to mutterstadt, 2.8 miles to mannheim (mhg)']",https://www.hotels.com/ho408334/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=2&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
2,Mutterstadt,Germany,"Radisson Blu Hotel, Mannheim",4.5,"Quadrant Q7, 27, Mannheim, 68161, Germany",387.0,Mannheim,130.0,"['6.4 miles to mutterstadt, 2.3 miles to mannheim (mhg)']",https://www.hotels.com/ho626280/?pa=3&tab=description&q-room-0-adults=2&intlid=SoldOutListing&ZSX=0&SYE=3&q-room-0-children=0
3,Mutterstadt,Germany,ACHAT Comfort Frankenthal/Pfalz,3.5,"Mahlastrasse 18, Frankenthal, RP, 67227, Germany",64.0,Frankenthal,83.0,"['5.9 miles to mutterstadt, 8.3 miles to mannheim (mhg)']",https://www.hotels.com/ho180949/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=4&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
4,Mutterstadt,Germany,Relax Apartment,4.0,"F4 14-15, Mannheim, 68159, Germany",175.0,Mannheim,130.0,"['6.2 miles to mutterstadt, 2.7 miles to mannheim (mhg)']",https://www.hotels.com/ho685840/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=5&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3


In [15]:
# Sanity check
trivago_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [57]:
# Most popular destination
most_popular_cities = trivago_df.city.value_counts().iloc[:100].index

In [58]:
most_popular_cities = list(most_popular_cities)

In [59]:
cities = []
for city in most_popular_cities:
    string = city.split(",")
    cities.append(string[0])
print(cities)

['London', 'Paris', 'Istanbul', 'New York', 'Rio de Janeiro', 'Amsterdam', 'Rome', 'Cancun', 'Tokyo', 'Berlin', 'Dubai', 'Madrid', 'Las Vegas', 'Barcelona', 'Prague', 'São Paulo', 'Natal', 'Florianópolis', 'Buenos Aires', 'Orlando', 'Bangkok', 'Kuala Lumpur', 'Miami Beach', 'Vienna', 'Porto Seguro', 'Milan', 'Foz do Iguaçu', 'Playa del Carmen', 'Osaka', 'Hamburg', 'Sydney', 'Armação dos Búzios', 'Melbourne', 'Maceió', 'Fortaleza', 'Porto de Galinhas', 'Acapulco', 'Lisbon', 'Singapore', 'Salvador Bahia', 'Puerto Vallarta', 'Gramado', 'Hong Kong', 'Malacca', 'Mexico City', 'Guarujá', 'Ubatuba', 'Munich', 'Venice', 'Budapest', 'Cartagena', 'Moscow', 'Edinburgh', 'Marrakech', 'Kyoto', 'Port Dickson', 'Campos do Jordão', 'Florence', 'Balneário Camboriú', 'Paraty', 'Antalya', 'Mazatlán', 'Caldas Novas', 'San Francisco', 'Dublin', 'Copenhagen', 'Cabo Frio', 'Mumbai', 'Cologne', 'Ankara', 'Guadalajara', 'Ilhabela', 'Seoul', 'Arraial do Cabo', 'Udaipur', 'Seville', 'Taipei City', 'St Petersburg

In [38]:
# Best reviews
best_reviews = hotels_df[hotels_df["rating"] >= 4.5]
best_reviews = best_reviews[best_reviews["popularity_rating"] > 50]

In [39]:
best_reviews

,city,country,hotel_name,rating,address,popularity_rating,locality,price,landmark,URL
2,Mutterstadt,Germany,"Radisson Blu Hotel, Mannheim",4.5,"Quadrant Q7, 27, Mannheim, 68161, Germany",387.0,Mannheim,130.0,"['6.4 miles to mutterstadt, 2.3 miles to mannheim (mhg)']",https://www.hotels.com/ho626280/?pa=3&tab=description&q-room-0-adults=2&intlid=SoldOutListing&ZSX=0&SYE=3&q-room-0-children=0
12,Mutterstadt,Germany,BoardingHouse Mannheim,4.5,"26 S6, Mannheim, 68161, Germany",53.0,Mannheim,108.0,"['6.5 miles to mutterstadt, 2.2 miles to mannheim (mhg)']",https://www.hotels.com/ho618455328/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=13&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
31,Mutterstadt,Germany,Qube Heidelberg,4.5,"Bergheimer Str. 74, Heidelberg, BW, 69115, Germany",595.0,Heidelberg,153.0,"['15 miles to mutterstadt, 8.6 miles to mannheim (mhg)']",https://www.hotels.com/ho335220/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=32&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
33,Mutterstadt,Germany,Hotel Europäischer Hof Heidelberg,4.5,"Friedrich-Ebert-Anlage 1, Heidelberg, BW, 69011, Germany",584.0,Heidelberg,251.0,"['16 miles to mutterstadt, 9.1 miles to mannheim (mhg)']",https://www.hotels.com/ho376787/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=34&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
55,Mutterstadt,Germany,Bacchus Hotel,4.5,"Rodensteinstrasse 30, Bensheim, 64625, Germany",158.0,Bensheim,134.0,"['21 miles to mutterstadt, 15 miles to mannheim (mhg)']",https://www.hotels.com/ho883080640/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=56&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
78,Passau,Germany,Landgasthof Euler,4.5,"Kaiserstraße 10, Neuschoenau, BY, 94556, Germany",52.0,Neuschoenau,88.0,"['21 miles to passau, 21 miles to passau central station']",https://www.hotels.com/ho1008295072/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=17&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
91,Hamburg,Germany,Hyperion Hotel Hamburg,4.5,"Amsinckstraße 39, Hamburg, 20097, Germany",677.0,Hammerbrook,116.0,"['0.9 miles to city center, 5.9 miles to hamburg airport (ham)']",https://www.hotels.com/ho534642/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=6&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
94,Hamburg,Germany,Clipper Elb-Lodge,4.5,"Carsten-Rehder-Strasse 71, Hamburg, HH, 22767, Germany",500.0,Hamburg-Altona,155.0,"['1.9 miles to city center, 6.6 miles to hamburg airport (ham)']",https://www.hotels.com/ho428977/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=9&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
95,Hamburg,Germany,Hotel Wedina an der Alster,4.5,"Gurlittstraße 23, Hamburg, HH, 20099, Germany",297.0,Saint Georg,143.0,"['0.8 miles to city center, 5.2 miles to hamburg airport (ham)']",https://www.hotels.com/ho509812/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=10&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
97,Hamburg,Germany,Hanse Clipper Haus,4.5,"Ditmar-Koel-Strasse 1, Hamburg, HH, 20459, Germany",712.0,Portuguese Quarter,153.0,"['0.7 miles to city center, 6.1 miles to hamburg airport (ham)']",https://www.hotels.com/ho428578/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=12&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3


In [40]:
best_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73048 entries, 2 to 426675
Data columns (total 10 columns):
city                 73048 non-null object
country              73048 non-null object
hotel_name           73048 non-null object
rating               73048 non-null float64
address              73048 non-null object
popularity_rating    73048 non-null float64
locality             73048 non-null object
price                73048 non-null float64
landmark             73048 non-null object
URL                  73048 non-null object
dtypes: float64(3), object(7)
memory usage: 6.1+ MB


In [41]:
# Sort best reviews by popularity rating and price
best_reviews = best_reviews.sort_values(by = ["popularity_rating"], axis = 0, 
                         ascending = False)

In [62]:
# Reset index
best_reviews = best_reviews.reset_index(drop = True)
best_reviews.head()

,city,country,hotel_name,rating,address,popularity_rating,locality,price,landmark,URL
0,Brunoy,France,Hôtel Henriette,4.5,"9 Rue des Gobelins, Paris, Paris, 75013, France",999.0,Paris,130.0,"['12 miles to brunoy, 15 miles to roissy - charles de gaulle airport (cdg)']",https://www.hotels.com/ho448748/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=36&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
1,Choisy-le-Roi,France,Hôtel Henriette,4.5,"9 Rue des Gobelins, Paris, Paris, 75013, France",999.0,Paris,130.0,"['5.8 miles to choisy-le-roi, 15 miles to roissy - charles de gaulle airport (cdg)']",https://www.hotels.com/ho448748/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=41&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
2,Göcek,Turkey,Yacht Classic Hotel - Boutique Class,4.5,"1 Karagozler No:24, Fethiye, Mugla, 48300, Turkey",999.0,Fethiye,130.0,"['13 miles to gocek, 12 miles to gocek marina']",https://www.hotels.com/ho376842/?pa=3&tab=description&q-room-0-adults=2&intlid=SoldOutListing&ZSX=0&SYE=3&q-room-0-children=0
3,Pinellas Park,United States,Barefoot Beach Club,4.5,"13238 Gulf Blvd, Madeira Beach, FL, 33708, United States",999.0,Madeira Beach,279.0,"['6.2 miles to pinellas park, 20 miles to tampa, fl (tpa-tampa intl.)']",https://www.hotels.com/ho247119/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=5&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
4,Madeira Beach,United States,Barefoot Beach Club,4.5,"13238 Gulf Blvd, Madeira Beach, FL, 33708, United States",999.0,Madeira Beach,279.0,"['1.0 mile to city center, 20 miles to tampa, fl (tpa-tampa intl.)']",https://www.hotels.com/ho247119/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=2&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3


In [100]:
# Unpersonalized recommendations
def hotel_recommendation_baseline():
    # Get list of most popular cities
    indexes = []
    for city in cities:
        # Get the hotels with 5.0 ratings that are at each of the top cities
        idxs = best_reviews[best_reviews["city"] == city].index
        indexes.append(idxs)
    flatten = functools.reduce(operator.iconcat, indexes, [])
    # Create dictionary to store results from each df index
    recommendation = {"city": [], "country": [], "hotel": [], "rating": [], "popularity": [], 
                     "price": [], "address": [], "landmark": [], "locality": [], "url": []}
    # Loop through all idxs in the flattened id list, and add the info at each idx row to dictionary
    for i, idx in enumerate(flatten):
        recommendation["city"].append(best_reviews.at[idx, "city"])
        recommendation["country"].append(best_reviews.at[idx, "country"])
        recommendation["hotel"].append(best_reviews.at[idx, "hotel_name"])
        recommendation["rating"].append(best_reviews.at[idx, "rating"])
        recommendation["popularity"].append(best_reviews.at[idx, "popularity_rating"])
        recommendation["price"].append(best_reviews.at[idx, "price"])
        recommendation["address"].append(best_reviews.at[idx, "address"])
        recommendation["landmark"].append(best_reviews.at[idx, "landmark"])
        recommendation["locality"].append(best_reviews.at[idx, "locality"])
        recommendation["url"].append(best_reviews.at[idx, "URL"])
    # Create df with dictionary
    baseline_recommendations = pd.DataFrame(recommendation)
    return baseline_recommendations

    
    

In [116]:
recommendations = hotel_recommendation_baseline()

In [117]:
# Sort recommendations 

recommendations = recommendations.sort_values(by = ["popularity"], axis = 0, 
                         ascending = False)

In [118]:
recommendations

,address,city,country,hotel,landmark,locality,popularity,price,rating,url
2,"9 Rue des Gobelins, Paris, Paris, 75013, France",Paris,France,Hôtel Henriette,"['1.2 miles to city center, 3.0 miles to eiffel tower']",Quartier de Croulebarbe,998.0,130.0,4.5,https://www.hotels.com/ho448748/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=133&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
3,"28 Place Des Vosges, Paris, Paris, 75003, France",Paris,France,Pavillon de la Reine,"['2.2 miles to left bank, 3.2 miles to eiffel tower']",Paris,994.0,328.0,4.5,https://www.hotels.com/ho122238/?q-check-out=2020-08-29&FPQ=5&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=173&tab=description&JHR=6&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
4,"9 rue de l'ancienne comedie, Paris, 75006, France",Paris,France,Hôtel Left Bank Saint Germain,"['1.2 miles to city center, 2.0 miles to eiffel tower']",Quartier Latin,982.0,251.0,4.5,https://www.hotels.com/ho563421/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=67&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
90,"Atocha 23, Madrid, Madrid, 28012, Spain",Madrid,Spain,Catalonia Puerta del Sol,"['0.2 miles to city center, 8.0 miles to madrid (mad-adolfo suarez madrid-barajas)']",Literary Quarter,975.0,108.0,4.5,https://www.hotels.com/ho292296/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=2&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
91,"Cuesta de Santo Domingo, 5, Madrid, Madrid, 28013, Spain",Madrid,Spain,Gran Meliá Palacio de los Duques-Leading Hotels of the World,"['0.4 miles to city center, 8.1 miles to madrid (mad-adolfo suarez madrid-barajas)']",Centro,975.0,295.0,5.0,https://www.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAD1T16rjWBD8m_uinXsUjoIHLouVZStn-WWRlbOVLX39mlnYly6qmoaC6iqX5TX_BmCZ4i1r43T-boY0rb6ToQPuH-2azk42bVWSgQ0CeViydv5_xbVV0vz9OU6aqi_4eIl_uO71a8cPEQkD1BpGnydpOoMaHfgrG1yFUbLNwm4N16TA_N7EkYQOw2TZDrEh6x2pBrBaF3NaU9iehnNgDoUpz-bKhfJRCLDsOzKxVvo9BfgDXctOf_Mgulec4XbSkGds2bG8VTb7hoV3j_R0BB0Lqx-4LQDlTC771Y8O4XxuRfOalSJ-H37-QlgWh5ZHQD_jHbZVho4w7RcjkGNUYVr3fJ1zgTS0m6sodWfxrT7BvEK9ucUnKMdprhiPIWDd1d5loOxgGgN6PjRu9RwtF1KVoQJaBOB26aF4KqndDlikDxb_cN0kDwaDwqwhs99Eox4M7KQ2L-y8r5kwVdkOSScewpruk6c7tlF7i4T02uEsBq7GkgdiSS03k7ewZKkAMwEDm2v9hll0kH4U2k7E92gqhohxixaTQSBE-2IirSNwYEz90O4Uon4uGbe2d4PJqofPyvoW4P0VwUJZ2WVDvusoRrjQ56bZ4cWNb_WFQx-Dpw1mARHtvjcEGrx6L6FxhiOb42TmEWM2BSqaQh3lLY11dUI-bkFz4kzr2fwjfrwT6hmkWvdmgVncXNuSDFJq4330lZhD-xkUoHryCCNmRk8H3nTrGpWoYoQsuYMn51ur6Px2TzBXfV-DjdTYNJ0CNy8XVfAaaiDBA4hmu8US_TYDqy9X50Hvz_UYqCIV9TdosU-e8nUncLV-pc9D11hxMK5dStTzwBmqGFmCocCTuu8mY2sCVZJPodS1Y3V4afq8bkMEdRjU_NkiCXUXQ53ayc7rw7S2KiCRQjSEYVi_Avyt...
5,"43, rue Monsieur le Prince, Paris, Paris, 75006, France",Paris,France,Hôtel Saint Paul Rive Gauche,"['1.0 mile to city center, 2.1 miles to eiffel tower']",Latin Quarter,954.0,171.0,4.5,https://www.hotels.com/ho288510/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=21&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
325,"Calle 93B N° 19-44, Bogotá, Distrito Capital, 110221, Colombia",San Francisco,Colombia,Casa Dann Carlton Hotel & Spa,"['24 miles to san francisco, 6.2 miles to bogota (bog-el dorado intl.)']",Bogotá,951.0,104.0,4.5,https://www.hotels.com/ho172373/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=9&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
224,"via del crocefisso del Lume 18, Florence, FI, 50126, Italy",Florence,Italy,Villa Olmi Firenze,"['3.0 miles to city center, 3.4 miles to florence santa maria novella station']",Quartiere 3,947.0,215.0,4.5,https://www.hotels.com/ho239265/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=171&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
361,"Morgado, 17 (e

In [119]:
recommendations.to_csv("../data/baseline_recommendations.csv")